<a href="https://colab.research.google.com/github/bylehn/auxetic_networks_jaxmd/blob/stress-response/auxetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
#!pip install jax-md
import jax.numpy as np
import numpy as onp
from jax import random
from jax.config import config; config.update("jax_enable_x64", True)
from jax_md import space
from jax import random, grad
from jax import jit, vmap
from jax import lax
import networkx as nx
import visualize
import utils
import simulation


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [7]:
steps = 50
write_every = 1
perturbation = 1.0
delta_perturbation = 0.1
number_of_nodes_per_side = 10

system = utils.System(number_of_nodes_per_side, 22, 2.0, 0.2, 1e-1)
system.initialize()
displacement = system.displacement
shift = system.shift
R = system.X

In [6]:
system.initial_angles

Array([1.737385  , 0.94250834, 0.79487665, ..., 1.20291301, 0.26466755,
       0.14759021], dtype=float64)

In [ ]:
steps = 50
write_every = 1
perturbation = 1.0
delta_perturbation = 0.1
#create graph

number_of_nodes_per_side = 10
N,G,X,E,bond_lengths = utils.createDelaunayGraph(number_of_nodes_per_side, 22, 2.0, 0.2, 1e-1)
R = np.array(X)
k_angle = 1e-1
k_bond, _ = utils.create_spring_constants(R,E,1.0)
surface_nodes = utils.getSurfaceNodes(G, number_of_nodes_per_side)
displacement, shift = space.free() #displacement = points in space, shift = small shifts of each particle

grad_f = grad(simulation.simulate_auxetic, argnums=1)

In [8]:
poisson, log, R_init, R_final = simulation.simulate_auxetic(R, system, shift, perturbation, delta_perturbation, displacement, steps, write_every, optimize=False)

ValueError: vmap in_axes specification must be a tree prefix of the corresponding value, got specification (None, None, 0, 0, None) for value tree PyTreeDef((*, *, *)).

In [ ]:
poisson

In [ ]:

N_trials=10
dw=0.2
w_c=2.0

ageing_rate=0.01
success_frac=0.05
freq_range=[w_c-dw/2,w_c+dw/2]
D_range = [x**2 for x in freq_range]

#create graph
number_of_nodes_per_side = 8
N,G,X,E,bond_lengths = utils.createDelaunayGraph(number_of_nodes_per_side, 25, 2.0, 0.4)
R = np.array(X)
k_angle = 1e-2
k_bond, _ = utils.create_spring_constants(R,E,1.0)
surface_nodes = utils.getSurfaceNodes(G, number_of_nodes_per_side)
displacement, shift = space.free() #displacement = points in space, shift = small shifts of each particle
angle_triplets_data = utils.calculate_angle_triplets(E)
theta_0 = utils.calculate_initial_angles(R, displacement, E, angle_triplets_data)
grad_f = grad(simulation.simulate_auxetic, argnums=1)
perturbation = 1
delta_perturbation = 0.1

M = simulation.get_mass(N, G)
m = np.ones(N)

k3,_,trial= simulation.optimizeAgeingCompression(R, M, m, G, N, w_c, dw, N_trials,ageing_rate,success_frac, k_bond, k_angle, shift, surface_nodes, perturbation, delta_perturbation, displacement, E, bond_lengths, theta_0, steps, write_every)